In [8]:
from gensim.models import Word2Vec, callbacks
import numpy as np

In [74]:
embed_incels = Word2Vec.load('../data/embeddings/embeddings/embeddings/incels_full.bin')
embed_feminism = Word2Vec.load('../data/embeddings/embeddings/embeddings/feminism_full.bin')
embed_braincels = Word2Vec.load('../data/embeddings/embeddings/embeddings/braincels.bin')
embed_fourthwavewomen = Word2Vec.load('../data/embeddings/embeddings/fourthwavewomen.bin')
embed_mensrights = Word2Vec.load('../data/embeddings/embeddings/embeddings/mensrights.bin')
embed_trufemcels = Word2Vec.load('../data/embeddings/embeddings/embeddings/trufemcels.bin')

In [68]:
embed_incels.wv.most_similar("woman", negative="man", topn=5)

[('foids', 0.47826236486434937),
 ('femoids', 0.44140955805778503),
 ('female', 0.440362811088562),
 ('females', 0.3937489688396454),
 ('they', 0.3356839418411255)]

In [71]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    # base_embed.init_sims(replace=True)
    # other_embed.init_sims(replace=True)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)

    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()  
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed

def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
        
    return (m1,m2)

In [75]:
# embed_incels_aligned = smart_procrustes_align_gensim(embed_feminism, embed_incels, words=None)
# embed_braincels_aligned = smart_procrustes_align_gensim(embed_feminism, embed_braincels, words=None)
embed_incels_aligned = smart_procrustes_align_gensim(embed_feminism, embed_incels, words=None)

25226 25226
25226 25226


In [76]:
embed_fourthwavewomen_aligned = smart_procrustes_align_gensim(embed_feminism, embed_fourthwavewomen, words=None)


12606 12606
12606 12606


In [80]:
embed_mensrights_aligned = smart_procrustes_align_gensim(embed_feminism, embed_mensrights, words=None)

12490 12490
12490 12490


In [77]:
embed_fourthwavewomen_aligned.wv.most_similar("woman", negative="man", topn=5)

[('historically', 0.35996028780937195),
 ('libfems', 0.35622909665107727),
 ('systemically', 0.34624382853507996),
 ('discrimination', 0.34381550550460815),
 ('overwhelmingly', 0.34236887097358704)]

In [78]:
embed_incels_aligned.wv.most_similar("woman", negative="man", topn=5)

[('femoids', 0.44140955805778503),
 ('female', 0.4403628408908844),
 ('females', 0.3937489986419678),
 ('they', 0.33568400144577026),
 ('mostly', 0.3343648910522461)]

In [81]:
embed_mensrights_aligned.wv.most_similar("woman", negative="man", topn=5)

[('men', 0.3657928705215454),
 ('female', 0.34590962529182434),
 ('historically', 0.3324694335460663),
 ('womens', 0.3239828646183014),
 ('overwhelmingly', 0.3086893856525421)]

In [79]:
embed_feminism.wv.most_similar("woman", negative="man", topn=5)

[('womens', 0.35674095153808594),
 ('worldwide', 0.3343164324760437),
 ('marginalized', 0.3183361887931824),
 ('people', 0.31121283769607544),
 ('marginalised', 0.3053872585296631)]

In [88]:
embed_fourthwavewomen_aligned.wv.most_similar("trans", topn=10)

[('cis', 0.743873119354248),
 ('afab', 0.6708109974861145),
 ('nb', 0.6152593493461609),
 ('transwomen', 0.604610800743103),
 ('inclusive', 0.6011746525764465),
 ('transmen', 0.5973383188247681),
 ('transgender', 0.5851062536239624),
 ('ci', 0.554002046585083),
 ('queer', 0.5521582365036011),
 ('lgbt', 0.5290417671203613)]

In [89]:
embed_feminism.wv.most_similar("trans", topn=10)

[('transgender', 0.773362934589386),
 ('nonbinary', 0.7053300738334656),
 ('nb', 0.6893020868301392),
 ('intersex', 0.6756387948989868),
 ('transwomen', 0.660694420337677),
 ('cis', 0.6591365337371826),
 ('afab', 0.6236242651939392),
 ('amab', 0.5751936435699463),
 ('transmen', 0.5613951683044434),
 ('queer', 0.5581699013710022)]

In [90]:
embed_incels_aligned.wv.most_similar("trans", topn=10)

[('transgender', 0.872365415096283),
 ('ftm', 0.6808600425720215),
 ('transgendered', 0.6708664298057556),
 ('mtf', 0.6565186977386475),
 ('transwomen', 0.6514958143234253),
 ('transsexual', 0.6390201449394226),
 ('transmen', 0.6373894810676575),
 ('transexual', 0.6355262398719788),
 ('cis', 0.6304329037666321),
 ('transgenders', 0.6283801794052124)]